In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os,sys,inspect
#sys.path.insert(0,"..")
os.chdir('..')

In [ ]:
!pwd

In [ ]:
import os 
import glob
import numpy as np
import torch
import matplotlib.pyplot as plt

from PIL import Image
from torchvision import transforms


try:
    from torchvision.transforms import InterpolationMode

    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC
    

In [ ]:
def to_numpy(tensor):
    return (
        tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    )


def _convert_image_to_rgb(image):
    return image.convert("RGB")


def preprocess_img(img):
    transform = transforms.Compose(
        [
            transforms.Resize(448, BICUBIC),
            transforms.CenterCrop(448),
            #_convert_image_to_rgb,
            transforms.ToTensor(),
        ]
    )
    return transform(img)

In [ ]:
scribbles_folder = './datasets/SCRIBBLES'
scribbles_paths = sorted(glob.glob(scribbles_folder + "/*.png"))[::-1][:1000] # For heavy masking [::-1] 
scribbles_paths[:5]
# https://github.com/hasibzunair/masksup-segmentation/blob/master/notebooks/exp_dataloader.ipynb

In [ ]:
scribble = Image.open(scribbles_paths[2]).convert('P')
scribble

In [ ]:
scribble_pre = preprocess_img(scribble)
scribble_pre

In [ ]:
scribble_pre.size

### Masked data loader experiment

In [ ]:
########## Reproducibility ##########
import random

random.seed(0)
np.random.seed(0)
os.environ["PYTHONHASHSEED"] = str(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [ ]:
from torch.utils.data import DataLoader


from pipeline.dataset import DataSetMaskSup

train_file = ["data/voc07/trainval_voc07.json"]
test_file = ["data/voc07/test_voc07.json"]
step_size = 4


train_dataset = DataSetMaskSup(train_file, ["randomflip", "resizedcrop"], 448, "voc07")
test_dataset = DataSetMaskSup(test_file, [], 448, "voc07")

train_loader = DataLoader(
    train_dataset, batch_size=16, shuffle=True, num_workers=8
)
test_loader = DataLoader(
    test_dataset, batch_size=16, shuffle=False, num_workers=8
)

In [ ]:
dt = next(iter(test_loader))
x = dt["img"]
xm = dt["masked_img"]
# xmo = dt["d"]
s = dt["scribble"]
y = dt["target"]

x.shape, xm.shape, y.shape

In [ ]:
import torch 

import numpy as np 
import matplotlib.pyplot as plt

# def to_img(ten):
#     ten =(ten[0].permute(1,2,0).detach().cpu().numpy()+1)/2
#     ten=(ten*255).astype(np.uint8)
#     return ten

def unnormalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

def to_img_(ten):
    curr_img = ten.detach().to(torch.device('cpu'))
    curr_img = unnormalize(curr_img,
                           torch.tensor([0, 0, 0]), # mean and std
                           torch.tensor([1, 1, 1])) 
    curr_img = curr_img.permute((1, 2, 0))
    curr_img = curr_img.detach().cpu().numpy()
    #print(np.unique(curr_img))
    curr_img = (curr_img * 255).astype(np.uint8)
    return curr_img


# a = to_img_(x[7])
# print(a.shape)
# plt.imshow(a)

In [ ]:
n_images = 5

fig, axes = plt.subplots(nrows=1, ncols=n_images, 
                         sharex=True, sharey=True, figsize=(10, 5))

for i in range(5):
    axes[i].imshow(to_img_(x[i]))
    axes[i].axis("off")

plt.savefig("./checkpoint/analysis/figures/images.pdf", facecolor="white", bbox_inches = 'tight', dpi=300)

In [ ]:
n_images = 5

fig, axes = plt.subplots(nrows=1, ncols=n_images, 
                         sharex=True, sharey=True, figsize=(10, 5))

for i in range(5):
    axes[i].imshow(to_img_(xm[i]))
    axes[i].axis("off")

plt.savefig("./checkpoint/analysis/figures/images_masked_old.pdf", facecolor="white", bbox_inches = 'tight', dpi=300)

In [ ]:
# n_images = 5

# fig, axes = plt.subplots(nrows=1, ncols=n_images, 
#                          sharex=True, sharey=True, figsize=(10, 5))

# for i in range(5):
#     axes[i].imshow(to_img_(xm[i]))
#     axes[i].axis("off")

# plt.savefig("./checkpoint/analysis/figures/images_masked.pdf", facecolor="white", bbox_inches = 'tight', dpi=300)

In [ ]:
a = to_img_(x[0])
plt.imshow(a)
plt.axis("off")
plt.savefig("./checkpoint/analysis/figures/img.png", facecolor="white", bbox_inches = 'tight', dpi=300)

In [ ]:
b = to_img_(xm[0])
plt.imshow(b)
plt.axis("off")
plt.savefig("./checkpoint/analysis/figures/masked_img.png", facecolor="white", bbox_inches = 'tight', dpi=300)

In [ ]:
c = to_img_(s[0])
plt.imshow(c, cmap='gray')
plt.axis("off")
plt.savefig("./checkpoint/analysis/figures/scribble.png", facecolor="white", bbox_inches = 'tight', dpi=300)

In [ ]:
import cv2 

im = c
row, col = im.shape[:2]
bottom = im[row-2:row, 0:col]
mean = cv2.mean(bottom)[0]
bordersize = 5
border = cv2.copyMakeBorder(
    im,
    top=bordersize,
    bottom=bordersize,
    left=bordersize,
    right=bordersize,
    borderType=cv2.BORDER_CONSTANT,
    value=[0, 0, 0]
)
plt.imshow(border, cmap="gray")
plt.axis("off")
plt.savefig("./checkpoint/analysis/figures/scribble.png", facecolor="white", bbox_inches = 'tight', dpi=300)

In [ ]:
# n_images = 5

# fig, axes = plt.subplots(nrows=1, ncols=n_images, 
#                          sharex=True, sharey=True, figsize=(10, 5))

# for i in range(5):
#     axes[i].imshow(to_img_(s[i]), cmap="gray")
#     axes[i].axis("off")

# plt.savefig("./checkpoint/analysis/figures/scribbles_thr.pdf", facecolor="white", bbox_inches = 'tight', dpi=300)

In [ ]:
# a = to_img_(s[0])
# np.unique(a)

In [ ]:
MASKED_PATH = "./datasets/SCRIBBLES"
MASKED_PATHS = sorted(glob.glob(MASKED_PATH + "/*.png"))[::-1][:1000]
len(MASKED_PATHS)

In [ ]:
import cv2 

def read_image(path):
    image = cv2.imread(path, -1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

images = []
for i in range(40):
    img = read_image(MASKED_PATHS[i])
    images.append(img)
    
images = np.array(images)
images.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_image_grid(images, ncols=None, cmap='gray'):
    '''Plot a grid of images'''
    if not ncols:
        factors = [i for i in range(1, len(images)+1) if len(images) % i == 0]
        ncols = factors[len(factors) // 2] if len(factors) else len(images) // 4 + 1
    nrows = int(len(images) / ncols) + int(len(images) % ncols)
    imgs = [images[i] if len(images) > i else None for i in range(nrows * ncols)]
    f, axes = plt.subplots(nrows, ncols, figsize=(3*ncols, 2*nrows))
    axes = axes.flatten()[:len(imgs)]
    for img, ax in zip(imgs, axes.flatten()): 
        if np.any(img):
            if len(img.shape) > 2 and img.shape[2] == 1:
                img = img.squeeze()
            ax.imshow(img, cmap=cmap)
            
            # Hide grid lines
            ax.grid(False)

            # Hide axes ticks
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            #f.tight_layout()
            plt.subplots_adjust(left=0.4,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.1, 
                    hspace=0.01)
    plt.savefig("./checkpoint/analysis/figures/masks.pdf", facecolor="white", bbox_inches = 'tight', dpi=300)

# make 16 images with 60 height, 80 width, 3 color channels
#images = np.random.rand(16, 60, 80, 3)

# plot them
plot_image_grid(images,  ncols=10)